### 미션

<aside>
📌

4가지 이상의 하이퍼파라미터를 조정하여 최적의 모델을 찾아봅시다.

- 최적의 파라미터 수치
- 해당 모델 평가 결과 : val_accuracy, val_loss
</aside>

### 하이퍼파라미터

- Unit size
- Kernel size
- Dropout rate
- Optimizer
- Learning rate
- Activation functions
- Batch size
- Regularization
- Strides
- Padding
- Conv, 풀링 Layer 갯수
- …등

In [9]:
!pip install keras-tuner

In [16]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import keras_tuner as kt

In [5]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [11]:
# 데이터 로드 및 전처리
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [19]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(32, 32, 3)))

    # Conv2D 레이어 추가
    for i in range(hp.Int("conv_layers", 2, 4)):  # 2~4개의 Conv 레이어
        model.add(
            layers.Conv2D(
                filters=hp.Choice(f"filters_{i}", [32, 64, 128, 256]),
                kernel_size=hp.Choice("kernel_size", [3, 5]),
                activation="relu",
            )
        )
        # MaxPooling2D에 padding="same" 추가 및 크기 확인
        if model.layers[-1].output_shape[1] >= 2 and model.layers[-1].output_shape[2] >= 2:
            model.add(layers.MaxPooling2D(pool_size=2, padding="same"))
        else:
            print("입력 크기가 너무 작아 MaxPooling2D를 생략합니다.")
            break

    model.add(layers.Flatten())

    # Dense 레이어 추가
    model.add(
        layers.Dense(
            units=hp.Int("dense_units", min_value=64, max_value=512, step=64),
            activation="relu",
        )
    )
    model.add(layers.Dropout(rate=hp.Float("dropout_rate", 0.2, 0.5, step=0.1)))
    model.add(layers.Dense(10, activation="softmax"))

    # 컴파일 설정
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4, 1e-5])
        ),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

# 베이지안 최적화 설정
tuner = kt.BayesianOptimization(
    hypermodel=build_model,  # 하이퍼모델 함수
    objective="val_accuracy",  # 최적화 목표
    max_trials=15,  # 최대 시도 횟수
    num_initial_points=3,  # 초기 랜덤 시도 수
    alpha=0.0001,  # 탐색 공간의 균형을 조절
    beta=2.6,  # 탐색 강도를 조절
    seed=42,  # 랜덤 시드
    directory="bayesian_tuning_cifar10",  # 결과 저장 디렉토리
    project_name="cifar10_bayesian",  # 프로젝트 이름
)

# 튜닝 시작
tuner.search(
    x_train,
    y_train,
    epochs=5,  # 한 번의 시도에서 학습할 최대 에포크
    validation_data=(x_test, y_test),
    callbacks=[keras.callbacks.EarlyStopping(patience=3)],  # 조기 종료 콜백
)

# 최적 하이퍼파라미터 가져오기
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("최적 하이퍼파라미터:")
for param, value in best_hps.values.items():
    print(f"{param}: {value}")

# 최적 모델 재학습
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(
    x_train,
    y_train,
    epochs=20,  # 최적 모델 재학습
    validation_data=(x_test, y_test),
    callbacks=[keras.callbacks.EarlyStopping(patience=3)],  # 조기 종료 콜백
)

# 테스트 데이터 평가
test_loss, test_acc = best_model.evaluate(x_test, y_test)
print(f"테스트 정확도: {test_acc}")
print(f"테스트 손실: {test_loss}")

Reloading Tuner from bayesian_tuning_cifar10/cifar10_bayesian/tuner0.json

Search: Running Trial #15

Value             |Best Value So Far |Hyperparameter
3                 |2                 |conv_layers
128               |128               |filters_0
5                 |3                 |kernel_size
64                |64                |filters_1
128               |256               |dense_units
0.4               |0.4               |dropout_rate
0.01              |0.001             |learning_rate
32                |64                |filters_2
32                |None              |filters_3



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 232, in _build_and_fit_model
    model = self._try_build(hp)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 164, in _try_build
    model = self._build_hypermodel(hp)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 155, in _build_hypermodel
    model

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 232, in _build_and_fit_model
    model = self._try_build(hp)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/src/engine/tuner.py", line 164, in _try_build
    model = self._build_hypermodel(hp)
  File "/usr/local/lib/python3.10/dist-packages/kera

In [21]:
# 하이퍼모델 정의
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(32, 32, 3)))

    # Conv2D 레이어 추가
    for i in range(hp.Int("conv_layers", 2, 4)):  # 2~4개의 Conv 레이어
        model.add(
            layers.Conv2D(
                filters=hp.Choice(f"filters_{i}", [32, 64, 128, 256]),
                kernel_size=hp.Choice("kernel_size", [3, 5]),
                activation="relu",
            )
        )
        model.add(layers.MaxPooling2D(pool_size=2))

    model.add(layers.Flatten())

    # Dense 레이어 추가
    model.add(
        layers.Dense(
            units=hp.Int("dense_units", min_value=64, max_value=512, step=64),
            activation="relu",
        )
    )
    model.add(layers.Dropout(rate=hp.Float("dropout_rate", 0.2, 0.5, step=0.1)))
    model.add(layers.Dense(10, activation="softmax"))

    # 컴파일 설정
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4, 1e-5])
        ),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

# 그리드 검색 기반 최적화 (Hyperband)
tuner = kt.Hyperband(
    hypermodel=build_model,
    objective="val_accuracy",
    max_epochs=10,  # 최대 에포크 수
    factor=3,       # 자원 분배 비율
    seed=42,
    directory="hyperband_tuning_cifar10",
    project_name="cifar10_hyperband",
)

# 튜닝 시작
tuner.search(
    x_train,
    y_train,
    epochs=10,
    validation_data=(x_test, y_test),
    callbacks=[keras.callbacks.EarlyStopping(patience=3)],
)

# 최적 하이퍼파라미터 출력
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("최적 하이퍼파라미터:")
for param, value in best_hps.values.items():
    print(f"{param}: {value}")

# 최적 모델 재학습
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(
    x_train,
    y_train,
    epochs=10,
    validation_data=(x_test, y_test),
    callbacks=[keras.callbacks.EarlyStopping(patience=3)],
)

# 평가
test_loss, test_acc = best_model.evaluate(x_test, y_test)
print(f"테스트 정확도: {test_acc}")
print(f"테스트 손실: {test_loss}")

Trial 24 Complete [00h 01m 37s]
val_accuracy: 0.6628999710083008

Best val_accuracy So Far: 0.6934999823570251
Total elapsed time: 00h 11m 34s
최적 하이퍼파라미터:
conv_layers: 2
filters_0: 128
kernel_size: 3
filters_1: 256
dense_units: 448
dropout_rate: 0.30000000000000004
learning_rate: 0.001
filters_2: 256
filters_3: 64
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.4172 - loss: 1.5981 - val_accuracy: 0.6303 - val_loss: 1.0587
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.6413 - loss: 1.0216 - val_accuracy: 0.6795 - val_loss: 0.9310
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.7066 - loss: 0.8359 - val_accuracy: 0.7069 - val_loss: 0.8443
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.7554 - loss: 0.6916 - val_accuracy: 0.7188 - val_loss: 0.8354
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.7975 - loss: 0.5736 - 

In [22]:
# 하이퍼모델 정의
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(32, 32, 3)))

    # Conv2D 레이어 추가
    for i in range(hp.Int("conv_layers", 2, 4)):  # 2~4개의 Conv 레이어
        model.add(
            layers.Conv2D(
                filters=hp.Choice(f"filters_{i}", [32, 64, 128, 256]),
                kernel_size=hp.Choice("kernel_size", [3, 5]),
                activation="relu",
            )
        )
        model.add(layers.MaxPooling2D(pool_size=2))

    model.add(layers.Flatten())

    # Dense 레이어 추가
    model.add(
        layers.Dense(
            units=hp.Int("dense_units", min_value=64, max_value=512, step=64),
            activation="relu",
        )
    )
    model.add(layers.Dropout(rate=hp.Float("dropout_rate", 0.2, 0.5, step=0.1)))
    model.add(layers.Dense(10, activation="softmax"))

    # 컴파일 설정
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4, 1e-5])
        ),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

# 랜덤 검색 (Random Search)
tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=15,  # 탐색 시도 횟수
    seed=42,
    directory="random_tuning_cifar10",
    project_name="cifar10_random",
)

# 튜닝 시작
tuner.search(
    x_train,
    y_train,
    epochs=10,
    validation_data=(x_test, y_test),
    callbacks=[keras.callbacks.EarlyStopping(patience=3)],
)

# 최적 하이퍼파라미터 출력
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("최적 하이퍼파라미터:")
for param, value in best_hps.values.items():
    print(f"{param}: {value}")

# 최적 모델 재학습
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(
    x_train,
    y_train,
    epochs=10,
    validation_data=(x_test, y_test),
    callbacks=[keras.callbacks.EarlyStopping(patience=3)],
)

# 평가
test_loss, test_acc = best_model.evaluate(x_test, y_test)
print(f"테스트 정확도: {test_acc}")
print(f"테스트 손실: {test_loss}")

Trial 15 Complete [00h 00m 03s]

Best val_accuracy So Far: 0.7390999794006348
Total elapsed time: 00h 14m 54s
최적 하이퍼파라미터:
conv_layers: 2
filters_0: 256
kernel_size: 5
filters_1: 256
dense_units: 512
dropout_rate: 0.2
learning_rate: 0.0001
filters_2: 64
filters_3: 128
Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 26s 15ms/step - accuracy: 0.3515 - loss: 1.7848 - val_accuracy: 0.5338 - val_loss: 1.3140
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.5607 - loss: 1.2455 - val_accuracy: 0.6071 - val_loss: 1.1167
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.6311 - loss: 1.0540 - val_accuracy: 0.6450 - val_loss: 0.9983
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.6755 - loss: 0.9420 - val_accuracy: 0.6826 - val_loss: 0.9135
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.7103 - loss: 0.8412 - val_accuracy: 0.6931 - val_loss: 0.8898
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.74

In [ ]:
def build_vgg16_model(hp):
    model = keras.Sequential()

    # VGG16의 특징: Conv 블록과 MaxPooling
    for i in range(3):  # VGG16의 기본 블록 3개
        model.add(
            layers.Conv2D(
                filters=hp.Choice(f"filters_block_{i}", [64, 128, 256]),
                kernel_size=3,
                padding="same",
                activation="relu",
            )
        )
        model.add(
            layers.Conv2D(
                filters=hp.Choice(f"filters_block_{i}", [64, 128, 256]),
                kernel_size=3,
                padding="same",
                activation="relu",
            )
        )
        model.add(layers.MaxPooling2D(pool_size=2, strides=2))

    model.add(layers.Flatten())

    # Fully Connected 레이어
    model.add(
        layers.Dense(
            units=hp.Int("dense_units", min_value=256, max_value=512, step=128),
            activation="relu",
        )
    )
    model.add(layers.Dropout(rate=hp.Float("dropout_rate", 0.2, 0.5, step=0.1)))

    # 출력 레이어
    model.add(layers.Dense(10, activation="softmax"))

    # 컴파일 설정
    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])
        ),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

# 튜너 설정 (랜덤 검색 또는 하이퍼밴드 중 선택)
tuner = kt.Hyperband(
    hypermodel=build_vgg16_model,
    objective="val_accuracy",
    max_epochs=10,  # 최대 에포크 수
    factor=3,       # 자원 분배 비율
    seed=42,
    directory="vgg16_tuning",
    project_name="cifar10_vgg16",
)

# 튜닝 시작
tuner.search(
    x_train,
    y_train,
    epochs=10,
    validation_data=(x_test, y_test),
    callbacks=[keras.callbacks.EarlyStopping(patience=3)],
)

# 최적 하이퍼파라미터 출력
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("최적 하이퍼파라미터:")
for param, value in best_hps.values.items():
    print(f"{param}: {value}")

# 최적 모델 재학습
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(
    x_train,
    y_train,
    epochs=20,
    validation_data=(x_test, y_test),
    callbacks=[keras.callbacks.EarlyStopping(patience=3)],
)

# 테스트 데이터 평가
test_loss, test_acc = best_model.evaluate(x_test, y_test)
print(f"테스트 정확도: {test_acc}")
print(f"테스트 손실: {test_loss}")

Trial 26 Complete [00h 04m 56s]
val_accuracy: 0.751800000667572

Best val_accuracy So Far: 0.7681000232696533
Total elapsed time: 00h 36m 32s

Search: Running Trial #27

Value             |Best Value So Far |Hyperparameter
256               |128               |filters_block_0
64                |64                |filters_block_1
256               |128               |filters_block_2
512               |384               |dense_units
0.3               |0.2               |dropout_rate
0.01              |0.001             |learning_rate
10                |10                |tuner/epochs
0                 |4                 |tuner/initial_epoch
0                 |2                 |tuner/bracket
0                 |2                 |tuner/round

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 43s 25ms/step - accuracy: 0.1016 - loss: 14.8041 - val_accuracy: 0.1000 - val_loss: 2.3047
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 73s 21ms/step - accuracy: 0.1011 - loss: 2.3042 - val_accuracy: 0.1000 - va

온10기/이국한/코어/서울
이번에 퀘스트를 진행하게 되면서 하이퍼파라미터 자동화 즉 튜너 기능에 대해 다뤄 볼 수 있었습니다. 다만 a-z까지 직접 코드를 입력하기보다는 어느정도 초안을 받아 튜너 종류를 바꿔서 사용해봤고 사용함에 따라 어떤 부분들이 수정되어 바뀌어나가는지 또 이렇게 탐색해 나가는 과정에서 앞에서 다루어 봤던 모델도 한번 적용해 보면서 좀 더 편해진 부분도 있지만 실질적으로 제대로 알고 쓰지 않으면 단순히 시간만 길어지고 비용만  늘어나게되는 부분도 많이 있다는 것을 알게되었습니다.



온10기/오병철/코어/경기
회고: 딥러닝 모델을 학습하면서 하이퍼파라미터 튜닝의 중요성을 깨달았고, 특히 학습률과 배치 크기가 모델 성능에 큰 영향을 미친다는 점이 인상적이었습니다. ResNet 구조를 배우면서 Skip Connection이라는 혁신적인 아이디어로 깊은 신경망의 학습 문제를 해결할 수 있다는 점이 흥미로웠습니다. 실제 모델 구현과 튜닝 과정에서 71%의 정확도를 달성했는데, 이론적 지식을 실제 구현으로 옮기는 과정에서 많은 시행착오와 배움이 있었습니다.